In [1]:
from PIL import Image

In [2]:
#read in image
user_input1_pic1 = Image.open('Pictures\\CDIConfigBatchPrepSettings.png')
user_input5_pic1 = Image.open('Pictures\\ui5_pic1.png')
user_input5_pic2 = Image.open('Pictures\\user_input5_pic2.png')
user_input6_pic1 = Image.open('Pictures\\user_input6_pic1.png')
user_input6_pic2 = Image.open('Pictures\\user_input5_pic2.png')
ui7_pic1 = Image.open('Pictures\\ui7_pic1.png')
ui7_pic2 = Image.open('Pictures\\ui7_pic2.png')
ui7_pic3 = Image.open('Pictures\\ui7_pic3.png')
ui8_pic1 = Image.open('Pictures\\ui8_pic1.png')
ui8_pic2 = Image.open('Pictures\\ui8_pic2.png')

In [48]:
def ColdFeedBatchPrep():
    while True:
        user_input = input("Do you know the file share directory? y/n or e to exit.")
        if user_input == 'y':
            r_files_in_dir()
            break
        elif user_input == 'n':
            print("Get the file share directory from the client and then re-run the program")
            break
        elif user_input == 'e':
            print("You are not exiting the program")
            break
        else:
            print("That was not one of the options, try again!")

In [50]:
ColdFeedBatchPrep()


            This is a cold feed issue. The document was processed by AXRM.
            *Tips:
                - The process for cold feed documents to get into Millennium:
                    - client or third party sends the documents to the directory that will process the documents
                    - AXRM will process the document and send the document to the upload application (ermx_auto or cdi_auto) listed in the report type properties setting.
                    - Once in the auto queue, the batch indexing service will process the documents.
                        - If successful, the document should appear in Millennium
                        - If batch indexing failed the document, you can find the document in cer_batchindex.
                - Each report type has it's own script that it uses to read the file names, and its own logs.
                    - You can usually find these scripts and logs in C:\ERMXData\Scripts and C:\ERMXData\RPWorking, respectively.
          

In [42]:
def r_files_in_dir():
    while True:
        user_input1 = input("""
                You need to figure out which application is processing these.
                There are two ways to figure out if Batch Prep or AXRM processed these documents.
                Go to the directory the client provided.
                Are there files ending in .error or .parse in the directory? y/n or e to exit.
                """)
        if user_input1 == 'y':
            user_input1_pic1.show()
            print("""
                    'In the screenshot:
                    Yellow highlight:
                      - This is the file directory.
                      - This should match with the file directory the client gave you earlier.
                    Blue highlight:
                      - This is how batch prep is going to read the files.
                      - The selection in the screenshot is for file names
                    Green highlight
                      - This is the filename format.
                      - The filename should be:
                        - batchclass_batchname_createdate_createtime_mrn_FIN_docalias_provider_trackingnbr'
                    """)
            print("""
                    Known Defects with Batch Prep:
                        – update the Source directory for Batch Prep on hsalbatch04. I am getting a message stating "Check the required fields".
                            -https://jira2.cerner.com/browse/DOCIMAGING-13599
                    """)

            break
        elif user_input1 == 'n':
            axrm_file_extension()
            break
        elif user_input1 == 'e':
            print("You are not exiting the program")
            break
        else:
            print("That was not one of the options, try again!")
        break

In [43]:
def batch_prep_filename_values():
    while True:
        user_input2 = input("""
        This is a batch prep issue.
        Find out where batch prep is running (ERMX, ACIS, or Batch server).
        Once you have found the server running Batch Prep,
            - open CDIConfig,
            - go to the services tab select the batch prep service, and open up the advanced settings

        Does the filename and Batch Prep settings in CDIConfig match? y/n or e to exit
        """)
        if user_input2 == 'y':
            print("""
            Something else is causing the files to fail.
            To figure out the reason, delete the .error at the end of the file and let batch prep reprocess the document.
                - Once you delete the .error, hit the start menu and search for 'msgviewwin' and open up the application.
                - When open, hit file -> load private file -> select cdi.mlg file.
                - When batch prep processes the document and it fails, you should see the error message in the cdi.mlg file and that will help you figure out what's going on with the service
            """)
            print("""
            Some error messages I've seen:
                - An unknown error (#5) has occurred querying service CDI_BatchIndexSvc. Access is denied
                    - https://dragondrop.cerner.com/615853-millennium-cerner-document-imaging-cdiconfig-access-denied-2
                - Access Denied
                    - Check the PDF's security settings; the settings could be preventing the prep service to process the document.
                - Wrong file format
                    - For example, I had a ticket where they sent word documents instead of PDFs. Batch Prep can't process word documents.
                - The batch preparation service failed to lock file '<some file name>_Endo_Cart\dmag.xml'
                    - SR 447015418
                        - Two folders were processed, and someone resent the same folders, causing the issue. When batch prep tried to process the duplicate folders, it couldn't rename the folder to .done since it already existed. i removed the duplicate folders, and the other batches started processing.
            """)
            break
        elif user_input2 == 'n':
            print("""
                            This is the issue.
                            Make sure the settings line up with the document filename being processed.
                            """)
            break
        elif user_input2 == 'e':
            print("You are not exiting the program")
            break
        else:
            print('Please enter y for yes, n for no, or e to exit program.')

In [44]:
def axrm_failedpdf():
    while True:
        user_input3_1 = input("""
            Go to ermx server and open AXRM (ApplicationXtenderReportsMgmtAdmin) and go to Report Processors -> local computer -> configuration -> report sources
            Do you see the directory in question inside AXRM? y/n or e to exit.
            """)
        if user_input3_1 == 'y':
            axrm_batchprep_main_issues()
            break
        elif user_input3_1 == 'n':
            batch_prep_filename_values()
            break
        elif user_input3_1 == 'e':
            print("You are not exiting the program")
            break
        else:
            print('Please enter y for yes, n for no, or e to exit program.')

In [45]:
def axrm_file_extension():
    while True:
        user_input3 = input("Are there files ending in .failedpdf or .cxpdf? y/n or e to exit.")
        if user_input3 == 'y':
            print("""
            This is a cold feed issue. The document was processed by AXRM.
            *Tips:
                - The process for cold feed documents to get into Millennium:
                    - client or third party sends the documents to the directory that will process the documents
                    - AXRM will process the document and send the document to the upload application (ermx_auto or cdi_auto) listed in the report type properties setting.
                    - Once in the auto queue, the batch indexing service will process the documents.
                        - If successful, the document should appear in Millennium
                        - If batch indexing failed the document, you can find the document in cer_batchindex.
                - Each report type has it's own script that it uses to read the file names, and its own logs.
                    - You can usually find these scripts and logs in C:\ERMXData\Scripts and C:\ERMXData\RPWorking, respectively.
                - ERMX can only process one directory at a time. If there are documents in different directories, ERMX will process one directory until all the files have been processed, before moving on to the next directory.
            *Useful wikis:
                - Helpful to learn how to set up the report types and any setting configuration. If documents are failing in AXRM, it is useful to go through the settings for the report type to make sure nothing is misconfigured.
                    - https://wiki.cerner.com/display/Content360/COLD+Feeds%3A+Build%2C+Configure%2C+Troubleshoot
                - Useful information to learn how to read the scripts that AXRM uses to process the documents.
                    - https://wiki.cerner.com/display/Content360/COLD+Feed+Scripting+Examples
        """)
            axrm_batchprep_main_issues()
            break
        elif user_input3 == 'n':
            axrm_failedpdf()
            break

In [46]:
def axrm_batchprep_main_issues():
    while True:
        try:
            user_input4 = int(input("""
                                    This is a cold feed issue. The document was processed by AXRM.
                                    *Tips:
                                    - The process for cold feed documents to get into Millennium:
                                        - client or third party sends the documents to the directory that will process the documents
                                        - AXRM will process the document and send the document to the upload application (ermx_auto or cdi_auto) listed in the report type properties setting.
                                        - Once in the auto queue, the batch indexing service will process the documents.
                                            - If successful, the document should appear in Millennium
                                            - If batch indexing failed the document, you can find the document in cer_batchindex.
                                    - Each report type has it's own script that it uses to read the file names, and its own logs.
                                        - You can usually find these scripts and logs in C:\ERMXData\Scripts and C:\ERMXData\RPWorking, respectively.
                                    - ERMX can only process one directory at a time. If there are documents in different directories, ERMX will process one directory until all the files have been processed, before moving on to the next directory.
                                    *Useful wikis:
                                    - Helpful to learn how to set up the report types and any setting configuration. If documents are failing in AXRM, it is useful to go through the settings for the report type to make sure nothing is misconfigured.
                                        - https://wiki.cerner.com/display/Content360/COLD+Feeds%3A+Build%2C+Configure%2C+Troubleshoot
                                    - Useful information to learn how to read the scripts that AXRM uses to process the documents.
                                        - https://wiki.cerner.com/display/Content360/COLD+Feed+Scripting+Examples

                                    What is the issue? Select:
                                    1 - Documents not making it into Millennium/Documents not in patient's chart.
                                    2 - Cold feed documents failing to cer_batchindex.
                                    3 - Duplicate cold feed documents in Millennium.
                                    0 - to exit
                                    """))
            if user_input4 == 1:
                print("""
                - If the document never made it to Millennium, start at the beginning of the process mentioned earlier.
                - Go to the file directory that the files were sent to.
                """)
                ui5()
            elif user_input4 == 2:
                print("""
                - To figure out why the documents failed, grab one of the blob handles, go to a Citrix server, open discernvisualdeveloper and search for the blob handle in the trans log or pending document table.
                    - Select *
                      From cdi_trans_log c
                      Where c.blob_handle = '[put blob handle here and erase the #1.00 at the end]*'
                      With uar_code(m,d)
                - Find the MAN_QUEUE_CAT_CD and MAN_QUEUE_ERR_CD fields to see the reason why the document failed.
                 """)
                print("""
                Common errors:
                    - DOS error:
                        - The date of service is not between the allotted time range set inside CDIConfig -> Event Validation.
                        -The screenshot below reads as followed
                            - The DOS sent with the document looks for the 'Discharge date' for the encounter, if that's not available, it will look for the 'admit date' and so on.
                        - If the document is failing due to DOS, then the DOS is outside the 720 days in the past or future from the discharge date.
                """)
                ui8_pic1.show()
                print("""
                - Encounter error:
                    - The batch indexing service couldn't match a patient with the MRN, and FIN.
                    - The MRN, FIN, or both are incorrect.
                    - To test, go to powerchart and search for the MRN.
                        - If a patient comes up, add the FIN number and search again.
                            - If a patient comes up, then make sure this is an active MRN and FIN.
                                - Tips:
                                    - MRNs inside of a paratheses (MRN), then its not an active MRN
                                    - I believe this goes for FINs also, but not 100%
                            - If a patient doesn't come up after you search for the MRN and FIN together, then search only using the FIN number
                                - If the FIN number doesn't find a patient, the FIN is wrong.
                            - If no patient comes up, then the MRN is wrong.
                - Versioning error:
                    - The document that was processed matched another document currently inside Millennium, which caused it to fail.
                    - In CDIConfig -> Event Validation, the 'Versioning Match Fields' section shows what the Batch Indexing service is searching for to see if this is a duplicate/different version of a document.
                """)
                ui8_pic2.show()
                break
            elif user_input4 == 3:
                print("""
                - There are a couple of defects with the old legacy architecture that cause duplicate documents.
                    1. AXRM or AX Release script is still writing a document to a CDI_AUTO application, Batch Index service may find the document, successfully index it and copy it to CEROTG, and create the Millennium references to it. But then the delete from the auto queue will fail because the document is still locked in the sending system.
                        - https://jira2.cerner.com/browse/DOCIMAGING-12278
                    2. Versioned documents with performing provider remain in Batch Index queue
                        - https://jira2.cerner.com/browse/DOCIMAGING-13271?filter=70688&jql=project%20in%20(PATESIG%2C%20EDM%2C%20DOCIMAGING)%20AND%20issuetype%20%3D%20Defect%20AND%20text%20~%20%22batch%20indexing%20auto%22%20ORDER%20BY%20created%20DESC
                    3. Auto-versioning fails if the incoming document doesn't have a blob handle and blob_uid is required in CEROTG
                        -https://jira2.cerner.com/browse/DOCIMAGING-10892
                """)
                break
            elif user_input4 == 0:
                print("You are exiting this program.")
                break
            else:
                print("That was not one of the options, try again!")
            break
        except:
            print("Invalid input. Please enter a number")

In [47]:
axrm_batchprep_main_issues()

Invalid input. Please enter a number
You are exiting this program.


In [28]:
def ui5():
    while True:
        user_input5 = input("Do you see the files ending with .failedpdf? y/n or e for exit.")
        if user_input5 == 'y':
            print("""
            The AXRM files failed.
            Find out why the docuemtns failed.
            Go to the report logs and select report type that is failing
            """)
            user_input5_pic1.show()
            print("In the report logs, open one of the failed logs to see the error")
            user_input5_pic2.show()
            print("""
            The error in the screenshot above:
                - ERROR	12/7/2022 16:55:20	INDEXING Validation error. Page 1: Invalid field value. Field: DATE_OF_SERVICE -- 15/47/
                - This error means that the file name format doesn't match the way the script is reading the filename. You will need to go look at the script and see what is wrong with the filename format.
                - https://dragondrop.cerner.com/1351526-cold-feed-failing-due-to-error-with-dos-but-dont-see-what-the-problem-is
            """)
            print("""
            Other possible error messages:
                - ERROR 8/2/2006 12:56:57 STORAGE This AX application is currently being modified by another user.
                    - https://dragondrop.cerner.com/98825-millennium-cerner-document-imaging-armx-not-processing-reports
                    - https://jira2.cerner.com/browse/DOCIMAGING-11231
                - ERROR5/18/2017 07:24:11INDEXINGNo index records created. Report processing failed
                    - https://dragondrop.cerner.com/339628-millennium-cerner-document-imaging-file-fails-to-process-in-cold-feed
                - ERROR 10/7/2019 12:49:40 UPLOAD Can't create PDF file. This operation is not permitted.
                    - https://dragondrop.cerner.com/1235754-axrm-cold-feed-unable-to-process-pdfs-with-security
                - ERROR	5/25/2022 09:45:28	INDEXING	Validation error. Page 1: Invalid field value. Field: TRACKING_NBR -- 10277729_10710963_20220517180201960_F_IRFPAI_F_IRFPAI
                    - The filename was being entered for the tracking number, but the filename was too big for the field, causing the documents to fail.
                    - We changed the tracking number variable from the filename to the MRN + Encounter, and it processed through successfully.
                - ERROR	3/29/2022 13:55:27	INDEXING	Incorrect application fields
                    - SR 442650237
                        - The 'BREEZE' report type was set to PFT_AUTO instead of CDI_ERMX, causing the data fields not to match up.
                        - I changed the upload database from PFT_AUTO to CDI_ERMX, updated the data fields, and documents started to process correctly
            """)
            break
        elif user_input5 == 'n':
            ui6()
            break
        elif user_input5 == 'e':
            print("You are exiting this program.")
            break
        else:
            print("That was not one of the options, try again!")

In [31]:
ui5()

You are exiting this program.


In [27]:
def ui6():
    while True:
        user_input6 = input("Do you see the files there with a .cxpdf ending? y/n or e for exit.")
        if user_input6 == 'y' or user_input6 == 'n':
            print("""
            - This means that the files were processed successfully.
            - Now that we know that the files processed through, we need to go to the next step of the cold feed process.
            - After the files get processed in AXRM, they are sent to the ERMX_AUTO, CDI_AUTO, or CEROTG queue, depending on if the client is single app architecture or the legacy architecture.
            - Go to the report types, right-click on the report type you've been working on and go to upload. You will see the queue that these documents were sent to.
            """)
            user_input6_pic1.show()
            print("Open up Document manager on the ERMX server and run a blank query for the auto queue the files are being sent to, to see if the files are in there.")
            user_input6_pic2.show()
            break
        elif user_input6 == 'e':
            print("You are exiting this program.")
            break
        else:
            print("That was not one of the options, try again!")

In [26]:
ui6()


            - This means that the files were processed successfully.
            - Now that we know that the files processed through, we need to go to the next step of the cold feed process.
            - After the files get processed in AXRM, they are sent to the ERMX_AUTO, CDI_AUTO, or CEROTG queue, depending on if the client is single app architecture or the legacy architecture.
            - Go to the report types, right-click on the report type you've been working on and go to upload. You will see the queue that these documents were sent to.
            
Open up Document manager on the ERMX server and run a blank query for the auto queue the files are being sent to, to see if the files are in there.

            - This means that the files were processed successfully.
            - Now that we know that the files processed through, we need to go to the next step of the cold feed process.
            - After the files get processed in AXRM, they are sent to the ERMX_AUTO, CDI_AUTO

In [23]:
def auto_man_queues():
    while True:
        user_input7 = input("Do you see the documents inside the AUTO queue? y/n or e for exit.")
        if user_input7 == 'y':
            print("""
            - If the documents in the AUTO queue, there are two main reasons that documents get stuck in the AUTO queue.
                1. The batch indexing service pointing to this AUTO queue isn't on, so the documents aren't processing.
                    - Go look for the batch indexing service pointing to this queue.
                    - You can find the batch indexing services two ways.
                        i. Go to the servers they might be on, which are the ACIS, or document imaging (batch) servers.
                        ii. Go to discernvisualdeveloper on a Citrix server and run this script.
                            - 'SELECT A.APPLCTX, A.APPLICATION_DIR, A.APPLICATION_IMAGE, A.APPLICATION_NUMBER
                                    , A.APPLICATION_STATUS, A.APPLICATION_VERSION
                                    , A.APP_CTX_ID, A.AUTHORIZATION_IND
                                    , A.CLIENT_NODE_NAME, A.CLIENT_START_DT_TM, A.CLIENT_TZ, A.DEFAULT_LOCATION
                                    , A.DEVICE_ADDRESS, A.DEVICE_LOCATION, A.END_DT_TM, A.LOGDIRECTORY
                                    , A.NAME, A.PARMS_FLAG, A.PERSON_ID, A_POSITION_DISP = UAR_GET_CODE_DISPLAY(A.POSITION_CD)
                                    , A.ROWID, A.START_DT_TM, A.TCPIP_ADDRESS, A.UPDT_APPLCTX
                                    , A.UPDT_CNT, A.UPDT_DT_TM, A.UPDT_ID
                                    , A.UPDT_TASK, A.USERNAME
                            FROM APPLICATION_CONTEXT A
                            WHERE A.APPLICATION_NUMBER = 4273000
                            WITH NOCOUNTER, SEPARATOR=" ", FORMAT, MAXREC = 100'
                    - Log into the server and make sure the service is on. open up the 'services' app, and CDIConfig.
                        - Inside the 'services' app, look to see if the batch indexing service is turned on and running with the cernerasp\<client nmemonic>cpdisvc account.
                            - Side note, some clients have a different cpdisvc account, such as tssocd-ssocdcpdisvc. It's not always cernerasp
                            - If it is running with correct log in info, this is not the issue.
            """)
            ui7_pic1.show()
            print("""
            - Inside CDIConfig go to services, and select batch indexing.
            - Make sure there is a batch indexing service pointing to the correct auto queue.
            """)
            ui7_pic2.show()
            print("Notice how in the services tab there are multiple batch indexing services. You can look at each of the indexing services' setting by hitting the dropdown and going through each service. The names line up with each other.")
            ui7_pic3.show()
            print("""
            2. There are a couple of defects with the old legacy architecture.
                i. AXRM or AX Release script is still writing a document to a CDI_AUTO application, Batch Index service may find the document, successfully index it and copy it to CEROTG, and create the Millennium references to it. But then the delete from the auto queue will fail because the document is still locked in the sending system.
                    - https://jira2.cerner.com/browse/DOCIMAGING-12278
                ii. Versioned documents with performing provider remain in Batch Index queue
                    - https://jira2.cerner.com/browse/DOCIMAGING-13271?filter=70688&jql=project%20in%20(PATESIG%2C%20EDM%2C%20DOCIMAGING)%20AND%20issuetype%20%3D%20Defect%20AND%20text%20~%20%22batch%20indexing%20auto%22%20ORDER%20BY%20created%20DESC
                iii. Auto-versioning fails if the incoming document doesn't have a blob handle and blob_uid is required in CEROTG
                    -https://jira2.cerner.com/browse/DOCIMAGING-10892
            """)
            break
        elif user_input7 == 'n':
            print("""
            - This means that the documents were processed by the batch indexing service.
                - The documents now will either be in the manual queue (legacy architecture) or CEROTG (single app architecture)
                - Processed successfully into Millennium.
            - To determine what happened,
                -In Document manager, run a blank query against all of the man queues to see if there are any documents inside of those queues.
            """)
            break
        elif user_input7 == 'e':
            print("You are exiting this program.")
            break
        else:
            print("That was not one of the options, try again!")

In [24]:
auto_man_queues()


            - This means that the documents were processed by the batch indexing service.
                - The documents now will either be in the manual queue (legacy architecture) or CEROTG (single app architecture)
                - Processed successfully into Millennium.
            - To determine what happened,
                -In Document manager, run a blank query against all of the man queues to see if there are any documents inside of those queues.
            


In [18]:
def ui8():
    while True:
        ui8 = input("Are there documents in the MAN queue? y/n or e to exit.")
        if ui8 == 'y':
            print("""
            - To figure out why the documents failed, grab one of the blob handles, go to a Citrix server, open discernvisualdeveloper and search for the blob handle in the trans log or pending document table.
                - Select *
                  From cdi_trans_log c
                  Where c.blob_handle = '[put blob handle here and erase the #1.00 at the end]*'
                  With uar_code(m,d)
            - Find the MAN_QUEUE_CAT_CD and MAN_QUEUE_ERR_CD fields to see the reason why the document failed.
            """)
            print("""
            Common errors:
                - DOS error:
                    - The date of service is not between the allotted time range set inside CDIConfig -> Event Validation.
                    -The screenshot below reads as followed
                        - The DOS sent with the document looks for the 'Discharge date' for the encounter, if that's not available, it will look for the 'admit date' and so on.
                    - If the document is failing due to DOS, then the DOS is outside the 720 days in the past or future from the discharge date.
            """)
            ui8_pic1.show()
            print("""
            - Encounter error:
                - The batch indexing service couldn't match a patient with the MRN, and FIN.
                - The MRN, FIN, or both are incorrect.
                - To test, go to powerchart and search for the MRN.
                    - If a patient comes up, add the FIN number and search again.
                        - If a patient comes up, then make sure this is an active MRN and FIN.
                            - Tips:
                                - MRNs inside of a paratheses (MRN), then its not an active MRN
                                - I believe this goes for FINs also, but not 100%
                        - If a patient doesn't come up after you search for the MRN and FIN together, then search only using the FIN number
                            - If the FIN number doesn't find a patient, the FIN is wrong.
                        - If no patient comes up, then the MRN is wrong.
            - Versioning error:
                - The document that was processed matched another document currently inside Millennium, which caused it to fail.
                - In CDIConfig -> Event Validation, the 'Versioning Match Fields' section shows what the Batch Indexing service is searching for to see if this is a duplicate/different version of a document.
            """)
            ui8_pic2.show()
            break
        elif ui8 == 'n':
            print("""
            - Go check the trans log and pending document table to see if you can find the batch.
                - If there are no transaction logging for this batch, check to see if the transaction logging is turned on in CDIConfig.
            - Go check PowerChart to see if you can find the document.
                - The document could be uploaded to a different day than it was processed,  so it can seem like the image isn't on the chart, but it is there, just not where the user was looking.
            """)
            break
        elif ui8 == 'e':
            print("You are exiting this program.")
            break
        else:
            print("That was not one of the options, try again!")

In [19]:
ui8()

That was not one of the options, try again!
That was not one of the options, try again!
That was not one of the options, try again!
You are exiting this program.
